# Importing packages

In [ ]:
import random
import warnings
import statistics
import numpy as np
import pandas as pd
import xgboost as xgb
import seaborn as sns
import plotly.io as pio
from statistics import mean
import plotly.express as px
from sklearn.svm import SVR
import plotly.graph_objs as go
from scipy.stats import cauchy
import matplotlib.pyplot as plt
from sklearn import linear_model
from sklearn.svm import LinearSVR
import plotly.graph_objects as go
from sklearn import preprocessing
import matplotlib.patches as mpatches
from sklearn.linear_model import Ridge
from datetime import datetime, timezone
from sklearn.linear_model import RidgeCV
from scipy.spatial.distance import cdist
from plotly.subplots import make_subplots
from pytz import timezone as pytz_timezone
from sklearn.linear_model import ElasticNet
from sklearn.ensemble import BaggingRegressor
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import MinMaxScaler
from IPython.core.display import display, HTML
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import TweedieRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import TimeSeriesSplit
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.ensemble import HistGradientBoostingRegressor

# Define functions

In [ ]:
def index_of_agreement (s, o):

    s,o = filter_nan(s,o)
    ia = 1 -(np.sum((o-s)**2))/(np.sum((np.abs(s-np.mean(o))+np.abs(o-np.mean(o)))**2))
    return ia


def filter_nan(s,o):

    s = np.array(s.copy())
    o = np.array(o.copy())
    data = np.array([s.flatten(),o.flatten()])
    data = np.transpose(data)
    data = data[~np.isnan(data).any(1)]
    s = data[:,0]
    o = data[:,1]
    return s, o

    print(relationship.mean)


def create_the_closest_set(pop_size, percent):
    points = np.zeros((pop_size, 2))
    step = 1/(pop_size-1)
    for i in range (0,pop_size):
        coef1 = step*i
        coef2 = 1 - coef1
        points[i][0] = coef1
        points[i][1] = coef2


    distance_matrix = cdist(points, points, 'euclidean')  

    neighbors = [np.argsort(distance_matrix[i]) for i in range(len(points))]

    for i, neighbor_indices in enumerate(neighbors):
        
        for neighbor_index in neighbor_indices:
            if neighbor_index != i:
                distance = distance_matrix[i, neighbor_index]
    index_to = int(pop_size*percent/100)+1
    df = pd.DataFrame(neighbors)
    df = df.iloc[:, 1:index_to]
    return df


def generate_indices(pop_size, A, p, i, set_of_indeces_for_mutation):
    index_set = set_of_indeces_for_mutation
    
    rnd_set = np.array(index_set.iloc[i])
    np.random.shuffle(rnd_set)
    print(rnd_set)
    return rnd_set[0], rnd_set[1],rnd_set[2]

def fitness_function(df, params, variable_type, n1, n2, MAE_norm, index_individual, pop_size, criterias, min_f1, max_f1, min_f2, max_f2):
    FEATURES = ['Temp', 'Cloud', 'Pressure', 'Rel-hum', 'Dew-point', 'Wind_cos', 'Wind_sin', 'Wind_sp',
                'Temp_lag_1', 'Temp_lag_2', 'Temp_lag_3', 'P_lag_1', 'P_lag_2', 'P_lag_3',
                'd_cos', 'd_sin', 'w_cos', 'w_sin', 'm_cos', 'm_sin']
    
    df.index = pd.to_datetime(df.index)

    df = df[(df.index.year == 2015) | (df.index.year == 2016) | (df.index.year == 2017)]

    X = df.drop(columns=['energy'])
    y = df['energy']
  
    feature_variables = params[n1:]
    new_features = []
    number_of_ones = 0
    validation_error = 0.0

    for i in range(0, len(feature_variables)):
        if feature_variables[i] == 1:
            new_features += [FEATURES[i]]
            number_of_ones += 1
    
    X = X[new_features]

    warnings.filterwarnings("ignore")

    for i in range(0, n1):
        if variable_type[i] == 'int':
            params[i] = int(params[i])

    colsample_bytree = params[0]
    learning_rate = params[1]
    max_depth = int(params[2])
    alpha = int(params[3])
    n_estimators = int(params[4])

    model = xgb.XGBRegressor(objective='reg:squarederror', colsample_bytree=colsample_bytree,
                             learning_rate=learning_rate, max_depth=max_depth, alpha=alpha, n_estimators=n_estimators)


    tscv = TimeSeriesSplit(n_splits=5)
    mae_scores = []
    weights = [0.0667, 0.1333, 0.2, 0.2667, 0.3333]  

    for fold_idx, (train_index, val_index) in enumerate(tscv.split(X)):
        X_train, X_val = X.iloc[train_index], X.iloc[val_index]
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_val = scaler.transform(X_val)

        y_train, y_val = y.iloc[train_index], y.iloc[val_index]

        model.fit(X_train, y_train)
        y_pred = model.predict(X_val)

        mae = mean_absolute_error(y_val, y_pred)
        weighted_mae = weights[fold_idx] * mae
        mae_scores.append(weighted_mae)

    performance = np.sum(mae_scores) 
    f1 = (performance - min_f1) / (max_f1 - min_f1)
    f2 = (number_of_ones - min_f2) / (max_f2 - min_f2)
    step = 1.0 / (pop_size - 1)
    coeff_1 = step * index_individual
    coeff_2 = 1.0 - coeff_1

    validation_error += coeff_1 * f1 + coeff_2 * f2

    if criterias == 0:
        return validation_error
    if criterias == 1:
        return f1, f2
    if criterias == 2:
        return performance


def init_population(population, pop_size, N, a, b, variable_type):
    for i in range(0, pop_size):
        for j in range (0, N):
            if (variable_type[j] == 'real'):
                population[i][j] = random.random()*(b[j]-a[j])+a[j]
            if (variable_type[j] == 'int'):
                population[i][j] = int(random.random()*(b[j]-a[j])+a[j])
            if (variable_type[j] == 'bool'):
                if (random.random()>0.5):
                    population[i][j] = 1
                else:
                    population[i][j] = 0

def generate_indices(pop_size, A, p, i, set_of_indeces_for_mutation):
    index_set = set_of_indeces_for_mutation
    
    rnd_set = np.array(index_set.iloc[i])
    np.random.shuffle(rnd_set)
    return rnd_set[0], rnd_set[1],rnd_set[2]

def borders (v, population, N, a, b, variable_type):
    for i in range(0, pop_size):
        for j in range (0, N):
            if (variable_type[j] == 'real'):
                if (v[i][j] > b[j]):
                    v[i][j] = (b[j]+population[i][j])/2
                if (v[i][j] < a[j]):
                    v[i][j] = (a[j]+population[i][j])/2
            if (variable_type[j] == 'int'):
                if (v[i][j] > b[j]):
                    v[i][j] = int((b[j]+population[i][j])/2)
                if (v[i][j] < a[j]):
                    v[i][j] = int((a[j]+population[i][j])/2)
def isNaN(num):
    return num != num

def calc_criterions (df, params,n1,n2):
    global_train_MAE = 0.0
    global_train_MSE = 0.0
    global_train_IA = 0.0
    global_train_R2 = 0.0
    
    global_val_MAE = 0.0
    global_val_MSE = 0.0
    global_val_IA = 0.0
    global_val_R2 = 0.0
    
    global_test_MAE = 0.0
    global_test_MSE = 0.0
    global_test_IA = 0.0
    global_test_R2 = 0.0
    
    data = df
    FEATURES = ['Temp','Cloud', 'Pressure','Rel-hum','Dew-point','Wind_cos','Wind_sin','Wind_sp',
          'Temp_lag_1','Temp_lag_2','Temp_lag_3','P_lag_1', 'P_lag_2','P_lag_3',
         'd_cos','d_sin','w_cos','w_sin','m_cos','m_sin']
    
    df.index = pd.to_datetime(df.index)

    df = df[(df.index.year==2015) | (df.index.year==2016)| (df.index.year==2017)]

    X = df.drop(columns=['energy'])
    y = df['energy']
  
   
    feature_variables = params[n1:]
    new_features = []
    number_of_ones = 0
    validation_error = 0.0
    for i in range(0, len(feature_variables)):
        if (feature_variables[i] == 1):
            new_features += [FEATURES[i]]
            number_of_ones = number_of_ones + 1
    
    X = X[new_features] 
    warnings.filterwarnings("ignore")

    for i in range(0, n1):
        if (variable_type[i] == 'int'):
            params[i] = int(params[i])

    colsample_bytree =params[0]
    learning_rate = params[1]
    max_depth = int(params[2])
    alpha = int(params[3])
    n_estimators = int(params[4])

    model = xgb.XGBRegressor(objective='reg:squarederror', colsample_bytree=colsample_bytree,
                              learning_rate=learning_rate, max_depth=max_depth, alpha=alpha, n_estimators=n_estimators)
    

    tscv = TimeSeriesSplit(n_splits=5)
    mae_scores_train = []
    mse_scores_train = []
    r2_scores_train = []
    ia_scores_train = []

    mae_scores_val = []
    mse_scores_val = []
    r2_scores_val = []
    ia_scores_val = []

    mae_scores_test = []
    mse_scores_test = []
    r2_scores_test = []
    ia_scores_test = []

    weights = [0.0667, 0.1333, 0.2, 0.2667, 0.3333]      
    for fold_idx, (train_index, val_index) in enumerate(tscv.split(X)):
        X_train, X_val = X.iloc[train_index], X.iloc[val_index]
        
        scaler = StandardScaler()
        
        X_train = scaler.fit_transform(X_train)
        
        X_val = scaler.transform(X_val)
        y_train, y_val = y[train_index], y[val_index]
                
        model.fit(X_train, y_train)


        y_pred = model.predict(X_train)

        mae = mean_absolute_error(y_train, y_pred) * weights[fold_idx]
        
        mse = mean_squared_error(y_train, y_pred) * weights[fold_idx]
        r2 = r2_score(y_train, y_pred) * weights[fold_idx]
        ia = index_of_agreement(y_train, y_pred) * weights[fold_idx]

        mae_scores_train.append(mae)
        mse_scores_train.append(mse)
        r2_scores_train.append(r2) 
        ia_scores_train.append(ia) 


        y_pred = model.predict(X_val)


        mae = mean_absolute_error(y_val, y_pred)
        weighted_mae = weights[fold_idx] * mae
        
        mse = mean_squared_error(y_val, y_pred)
        weighted_mse = weights[fold_idx] * mse
        
        r2 = r2_score(y_val, y_pred)
        weighted_r2 = weights[fold_idx] * r2
        
        ia = index_of_agreement(y_val, y_pred)
        weighted_ia = weights[fold_idx] * ia
        

        mae_scores_val.append(weighted_mae)
        mse_scores_val.append(weighted_mse)
        r2_scores_val.append(weighted_r2) 
        ia_scores_val.append(weighted_ia)

    
    df.index = pd.to_datetime(df.index)

    df = data

    df_train = df[(df.index.year==2015) | (df.index.year==2016)| (df.index.year==2017)]
    df_test = df[(df.index.year==2018)]

    X_train = df_train.drop(columns=['energy'])
    X_train = X_train[new_features]
    
    y_train = df_train['energy']
    X_test = df_test.drop(columns=['energy'])
    X_test = X_test[new_features]
    y_test = df_test['energy']
    
    
        
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
 
    
    model.fit(X_train, y_train)
    
    y_pred = model.predict(X_test)

    global_train_MAE = np.sum(mae_scores_train)
    global_train_MSE =np.sum(mse_scores_train)
    global_train_R2 = np.sum(r2_scores_train)
    global_train_IA = np.sum(ia_scores_train)

    global_val_MAE = np.sum(mae_scores_val)
    global_val_MSE = np.sum(mse_scores_val)
    global_val_R2 = np.sum(r2_scores_val)
    global_val_IA = np.sum(ia_scores_val)

        
    global_test_MAE = mean_absolute_error(y_test, y_pred)
    global_test_MSE = mean_squared_error(y_test, y_pred)
    global_test_R2 = r2_score(y_test, y_pred)
    global_test_IA = index_of_agreement(y_test, y_pred)
        
    
    return global_train_MAE, global_train_MSE, global_train_R2, global_train_IA, global_val_MAE,global_val_MSE,global_val_R2,global_val_IA,global_test_MAE,global_test_MSE,global_test_R2,global_test_IA


In [ ]:
pop_size = 100
n1 = 5
n2 = 20
N = n1 + n2
pop_size = 100

df = pd.read_csv("data_daily.csv", index_col=0)
set_of_indeces_for_mutation = create_the_closest_set(pop_size, 10)


RUNS = 5
columns = ["colsample_bytree","learning_rate", "max_depth", "alpha","n_estimators",
        'Temp','Cloud', 'Pressure','Rel-hum','Dew-point','Wind_cos','Wind_sin','Wind_sp',
          'Temp_lag_1','Temp_lag_2','Temp_lag_3','P_lag_1', 'P_lag_2','P_lag_3',
         'd_cos','d_sin','w_cos','w_sin','m_cos','m_sin']
df_stats = pd.DataFrame()

variable_type =  ['real','real','int','int','int',
                  'bool','bool','bool','bool','bool','bool','bool','bool','bool','bool',
                  'bool','bool','bool','bool','bool','bool','bool','bool','bool','bool',
                  ]
a =[0.001, 0.001,1,1,1]
b = [1,   1,   20,10,300]

H = 10

best_in_RUN = []
fitness_stat = pd.DataFrame()
solution_stat = []

while (RUNS>0):

    global_best = 1e300
    population = np.empty((pop_size,N))
    v = np.empty((pop_size,N))

    fitness = np.empty(pop_size)
    fitness_new = np.empty(pop_size)
    F_history = np.empty(H)
    CR_history = np.empty(H)
    S_CR = []
    S_F = []
    w = []
    r = np.empty(pop_size)
    CR = np.empty(pop_size)
    F = np.empty(pop_size)
    p = np.empty(pop_size)

    FEV = 5000
    kratnost = FEV/100
    tracking_fitness = []
    tracking_solution = []
    init_population(population, pop_size, N, a, b, variable_type)
    MAE_norm = 0
    find_max_mae = []
    
    for i in range (0, pop_size):
        if (int(sum(population[i][n1:])) == 0):
            population[i][n1:n1+random.randint(0, n2)]=1
        print(i, ' ')
        x = population[i][:]
        solution = x.copy()
        #print(i, " ", solution)
        mae = calc_criterions (df, x,n1,n2)[4]
        find_max_mae.append(mae)   
        print(find_max_mae)
        print(np.max(find_max_mae))
        
    f1_max = np.max(find_max_mae)
    f1_min = 0.0
    f2_max = n2
    f2_min = 1
    
    
    for i in range (0, pop_size):
        if (int(sum(population[i][n1:])) == 0):
            population[i][n1:n1+random.randint(0, n2)]=1

        x = population[i][:]
        solution = x.copy()
        
        fit = fitness_function(df, x, variable_type ,n1, n2, 2.5, i, pop_size,0, min_f1,max_f1,min_f2,max_f2)
        fitness[i] = fit
    

        fitness_new[i] = fitness[i]
        FEV =FEV - 1

        if (fitness[i]<global_best):
            global_best = fitness[i]
            best_solution = solution.copy()
            print("GLOVAL BEST, FEV: ", FEV,": ", global_best, ", best solution: ", best_solution )

        if (FEV % kratnost == 0):
            tracking_fitness.append(global_best)
            tracking_solution = (best_solution)

    F_history[:] = 0.5
    CR_history[:] = 0.5
    A = 0
    k=0
    pmin = 5.0/pop_size
    MAE_norm = min(fitness)
    print(MAE_norm)
    while (FEV>0):
        CR_df = pd.DataFrame(pd.DataFrame(CR_history).T)

        S_CR = np.empty(pop_size)
        S_F = np.empty(pop_size)
        v[:][:]=population[:][:]
        for i in range (0, pop_size):
            r[i] = int(random.random()*H)
            CR[i] = np.random.normal(CR_history[int(r[i])], 0.1)
            if CR[i]>1:
                CR[i] = 1
            if CR[i]<0:
                CR[i] = 0

            F[i] = F_history[int(r[i])]+np.random.standard_cauchy()*0.1
            if F[i]>1:
                F[i] = 1
            while F[i]<0:
                F[i] = F_history[int(r[i])]+np.random.standard_cauchy()*0.1
            p[i] = random.random()*(0.2-pmin)+pmin

            r1, r2, r3 = generate_indices(pop_size, A, p, i, set_of_indeces_for_mutation)

            j_rand = int(random.random()*N)
            for j in range (0,n1):
                if (random.random()<CR[i] or j == j_rand):
                    v[i][j] =population[i][j]+F[i]*(population[r3][j]-population[i][j])+F[i]*(population[r1][j]-population[r2][j])


            for j in range (n1,n1 + n2):

                if (random.random()<CR[i] or j == j_rand):
                    rnd_bool_var = pop_size+1
                    var_list = [i,r1, r2, r3]
                    while (rnd_bool_var >= pop_size):  
                        rnd_bool_var = random.choice(var_list)

                    v[i][j] = population[rnd_bool_var][j]


            if (random.random()<(1.0/(n2))):
                rnd_ind =random.randint(n1, n1 + n2-1) 
                if (v[i][rnd_ind] == 0):
                    v[i][rnd_ind] =1
                else:
                     v[i][rnd_ind] =1

            for j in range (0,n1):
                if ( (isNaN(v[i][j]) == 1)):
                    if variable_type[j] == 'real':
                        v[i][j] = random.uniform(a[j], b[j])
                    if variable_type[j] == 'int':
                        v[i][j] = random.randint(int(a[j]), int(b[j]))

            chech_sum_ones = 0
            for j in range (n1,n1 + n2):
                if (v[i][j] == 1):
                    chech_sum_ones = chech_sum_ones +1
            if (chech_sum_ones == 0):
                v[i][random.randint(n1, n1 + n2-1)] = 1
            borders (v, population, N, a, b, variable_type)



        w = np.array([])
        S_CR = np.array([])
        S_F = np.array([])
        for i in range (0, pop_size):

            solution = v[i][:]
            x = solution = v[i][:]
            fit = fitness_function(df, x, variable_type ,n1, n2, 2.5, i, pop_size,0, min_f1,max_f1,min_f2,max_f2)
            fitness_new[i] = fit
            FEV = FEV-1
            if (fitness_new[i]<fitness[i]):
                S_CR = np.append(S_CR, CR[i])
                S_F = np.append(S_F, F[i])
                w = np.append(w,(fitness[i]-fitness_new[i]))

                population[i][:] = solution

                fitness[i] =fitness_new[i]

                if (fitness[i]<global_best):
                    global_best = fitness[i]
                    best_solution = solution.copy()


            if (FEV % kratnost == 0):
                tracking_fitness.append(global_best)
                tracking_solution = (best_solution)
                print("GLOVAL BEST, FEV: ", FEV,": ", global_best, ", best solution: ", best_solution )
        total_w = np.sum(w)
        w = w/total_w

        new_CR = np.sum(w*S_F)
        new_F = np.sum(w*S_F*S_F)/np.sum(w*S_F)
        if (new_CR >0 and new_F>0):
            F_history[k]=new_F
            CR_history[k]=new_CR
            k=k+1
            if (k>H-1):
                k=0

    print(str(RUNS))
    RUNS = RUNS - 1
    tracking_fitness = pd.DataFrame(tracking_fitness)

    fitness_stat = pd.concat([fitness_stat,tracking_fitness], axis=1)
    solution_stat.append(tracking_solution)

    best_in_RUN.append(global_best)
    df_stats = pd.concat([df_stats, pd.DataFrame(population)])
    print(df_stats)
    print(tracking_solution)


df_stats.columns = columns
df_stats.to_csv('XGBoost_D_ALL.csv')
